# Análise de Proteínas

## 1. Processamento do arquivo original

### Importação das libs

In [1]:
from Bio import SeqIO
import pandas as pd
import re

### Processamento do Arquivo

In [2]:
arquivo = "proteinas.fa"

headers = []
seqs = []
classes = []

for registro in SeqIO.parse(arquivo, "fasta"):
    headers.append(registro.id)
    seqs.append(str(registro.seq))    
    classes.append(registro.description)

### DataFrame

In [3]:
data_proteinas = pd.DataFrame({
    'header': headers,
    'sequencia': seqs,
    'classe_scop': classes
})

print(f'Total de proteínas: {len(data_proteinas)}')
print(f'Comprimento medio: {data_proteinas['sequencia'].str.len().mean():.2f} aminoácidos.')
print(f'Menor comprimento: {data_proteinas['sequencia'].str.len().min()} aminoácidos.')
print(f'Maior comprimento: {data_proteinas['sequencia'].str.len().max()} aminoácidos.')

print()
print('Primeiras 5 entradas:')
data_proteinas.head()

Total de proteínas: 15177
Comprimento medio: 184.82 aminoácidos.
Menor comprimento: 20 aminoácidos.
Maior comprimento: 1664 aminoácidos.

Primeiras 5 entradas:


,header,sequencia,classe_scop
0,d1dlwa_,slfeqlggqaavqavtaqfyaniqadatvatffngidmpnqtnkta...,d1dlwa_ a.1.1.1 (A:) Protozoan/bacterial hemog...
1,d2gkma_,gllsrlrkrepisiydkiggheaievvvedffvrvladdqlsaffs...,d2gkma_ a.1.1.1 (A:) Protozoan/bacterial hemog...
2,d1ngka_,ksfydavggaktfdaivsrfyaqvaedevlrrvypeddlagaeerl...,d1ngka_ a.1.1.1 (A:) Protozoan/bacterial hemog...
3,d2bkma_,eqwqtlyeaiggeetvaklveafyrrvaahpdlrpifpddltetah...,d2bkma_ a.1.1.1 (A:) automated matches {Geobac...
4,d4i0va_,aslyeklggaaavdlavekfygkvladervnrffvntdmakqkqhq...,d4i0va_ a.1.1.1 (A:) automated matches {Synech...


## 2. Extração das classes SCOP

In [4]:
scop_padrao = r'([a-z]\.[0-9]+\.[0-9]+\.[0-9]+)'

### Método para extração e aplicação

In [5]:
def extrair_classe_scop(descricao):
    match = re.search(scop_padrao, descricao)
    if match:
        return match.group(1)
    else:
        return None
    
data_proteinas['classe_scop'] = data_proteinas['classe_scop'].apply(extrair_classe_scop)

### Visualização dos resultados:
Resumo da Extração:

In [6]:
resumo_extracao = pd.DataFrame({
    'Metrica':[
        'Total de Sequências',
        'Com classe SCOP identificada',
        'Sem classe SCOP',
        'Taxa de Sucesso',
        'Classes únicas identificadas'
    ],
    'Valor':[
        len(data_proteinas),
        data_proteinas['classe_scop'].notna().sum(),
        data_proteinas['classe_scop'].isna().sum(),
        f'{(data_proteinas['classe_scop'].notna().sum()/len(data_proteinas)*100):.2f}%',
        data_proteinas['classe_scop'].nunique()
    ]        
})

resumo_extracao.head()

,Metrica,Valor
0,Total de Sequências,15177
1,Com classe SCOP identificada,15177
2,Sem classe SCOP,0
3,Taxa de Sucesso,100.00%
4,Classes únicas identificadas,4703


Distribuição das Classes:

In [7]:
distribuicao_classes = data_proteinas['classe_scop'].value_counts().head()
df_distrubuicao = pd.DataFrame({
    'Classe SCOP': distribuicao_classes.index,
    'Quantidade': distribuicao_classes.values,
    'Percentual': (distribuicao_classes.values / len(data_proteinas)*100).round(2)
})

df_distrubuicao

,Classe SCOP,Quantidade,Percentual
0,c.2.1.0,196,1.29
1,c.94.1.0,132,0.87
2,b.1.1.0,127,0.84
3,c.47.1.0,121,0.80
4,c.37.1.0,109,0.72


Amostra dos dados com classes extraídas:

In [8]:
amostra = data_proteinas[['header', 'classe_scop', 'sequencia']].head(11).copy()
amostra['comprimento'] = amostra['sequencia'].str.len()

amostra

,header,classe_scop,sequencia,comprimento
0,d1dlwa_,a.1.1.1,slfeqlggqaavqavtaqfyaniqadatvatffngidmpnqtnkta...,116
1,d2gkma_,a.1.1.1,gllsrlrkrepisiydkiggheaievvvedffvrvladdqlsaffs...,127
2,d1ngka_,a.1.1.1,ksfydavggaktfdaivsrfyaqvaedevlrrvypeddlagaeerl...,126
3,d2bkma_,a.1.1.1,eqwqtlyeaiggeetvaklveafyrrvaahpdlrpifpddltetah...,128
4,d4i0va_,a.1.1.1,aslyeklggaaavdlavekfygkvladervnrffvntdmakqkqhq...,123
5,d1asha_,a.1.1.2,anktrelcmkslehakvdtsnearqdgidlykhmfenypplrkyfk...,147
6,d2dc3a_,a.1.1.2,eelseaerkavqamwarlyancedvgvailvrffvnfpsakqyfsq...,172
7,d4hswa_,a.1.1.2,gfkqdiatirgdlrtyaqdiflaflnkypderryfknyvgksdqel...,137
8,d1ecaa_,a.1.1.2,lsadqistvqasfdkvkgdpvgilyavfkadpsimakftqfagkdl...,136
9,d1x9fd_,a.1.1.2,eclvteslkvklqwasafghahervafglelwrdiiddhpeikapf...,140


Estatísticas Extras:

In [12]:
distribuicao_classes = data_proteinas['classe_scop'].value_counts()

estatisicas_classes = pd.DataFrame({
    'Estatística': [
        'Média de sequências por classe',
        'Classe com mais sequências',
        'Classe com menos sequências',
        'Classes com apenas uma sequência'
    ],
    'Valor': [
        f'{data_proteinas['classe_scop'].value_counts().mean():.1f}',
        f"{distribuicao_classes.index[0]} ({distribuicao_classes.iloc[0]} sequências)",
        f'{data_proteinas['classe_scop'].value_counts().min()}',
        f'{(data_proteinas['classe_scop'].value_counts() == 1).sum()}'
    ]
})

estatisicas_classes

,Estatística,Valor
0,Média de sequências por classe,3.2
1,Classe com mais sequências,c.2.1.0 (196 sequências)
2,Classe com menos sequências,1
3,Classes com apenas uma sequência,2454


# 3. Implementação do K-mer 2x2